In [6]:
from openpyxl import load_workbook
import pandas as pd
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows
from pretty_html_table import build_table
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import time

In [12]:
path = r'C:\Users\Admin\Email_purchase_request\Input_data.xlsx'  #Path to the input excel file with data
wb = load_workbook(path)
ws = wb.worksheets[0]

data = ws.values
columns = next(data)[0:]
df = pd.DataFrame(data, columns=columns)

contacts = {'ABB': None, 'Chint': 'info@chint.ru',     # recipient addresses
            'WAGO': 'info@wago.com', 'KEAZ': None}

# Get the list of purchase requests
requests = df['p_request'].value_counts()

# схлопывание одинаковых по кол-ву
df = df.groupby(['item_name'], as_index=False).agg(
    {'manufacturer': 'first', 'part_no': 'first', 'quant': 'sum', 'unit': 'first'})[
    ['manufacturer', 'part_no', 'item_name', 'quant', 'unit']].sort_values('manufacturer')

# Divides the data into tables regarding to the supplier for further sending  
# ДItems for which there’s no supplier information are replacing to the new sheet in the excel book 
wb.create_sheet('result', 1) # Creating a new sheet in the book 
ws1 = wb['result']        
        
for i in df['manufacturer'].unique():
    data = (df[df['manufacturer'] == str(i)].sort_values(['item_name'] ,ascending=[True]))
    table = build_table(data, 'blue_light')
    try:
        send_emails(table, contacts[str(i)])
    except:    
        for r in dataframe_to_rows(data, index=False, header=True):
            ws1.append(r)

# The following excel sheet contains the list of request that are not marked with a supplier yet
wb.create_sheet('requests', 2) 
ws2 = wb['requests']            
requests = np.array(requests.index.tolist()).reshape(len(requests), 1)
for r in requests:
    ws2.append(tuple(r))
        
wb.save(path) 


sent
sent


In [2]:
# Send email function. Receives the letter body as an imput 
# Requires login and password 

def send_emails(tbl, adrs):

        for mail in [adrs,]:
            my_email = 'sender@email.com'
            server = smtplib.SMTP('smtp.gmail.com', 587) #587 - port 
            server.starttls()
            time.sleep(1)
            server.login(my_email, "password")  # Gain access 

            msg = MIMEMultipart()  # Create message 
            msg['From'] = my_email  # Sender 

            msg['Subject'] = 'Запрос счета'  # Message theme 
            msg['Return-Receipt-To'] = my_email

            msg['To'] = mail  # Receiver 
            body = ("<!DOCTYPE html><head></head><body>Hello, dear " + mail + 
                    '. Could you please send me your commercial offer for:' + "</body></html>")
            msg.attach(MIMEText(body, 'html'))  # Add the text in the message 
            msg.attach(MIMEText(tbl, 'html'))
            try:
                server.send_message(msg)  # Sending the message 
            except Exception:
                print("error ", mail)

            else:
                print("sent")

            time.sleep(40) # Delay to avoid ban 

            server.quit()  # Exit
